In [2]:
import pandas as pd
import numpy as np
import pytz
import plotly 
import geopy
import plotly.express as px
import folium
import matplotlib.pyplot as plt
import datetime

from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [177]:
from random import randint
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)
    

In [178]:
rnb = []
for i in range(30):
    rnb.append(random_with_N_digits(9))

In [59]:
#orders_raw = pd.read_csv('2023-2028_forecasting_form.csv')
FC_ZIP = pd.read_csv('FC_zip.csv').drop(axis = 1, columns = ['Unnamed: 0'])
GA_wkly = pd.read_csv('GA_weekly.csv')

In [60]:
GA_wkly

,Unnamed: 0,Week,Product,product category,sub-category,Variance,Quantity
0,0,1,1,A,f,31.125000,515
1,1,1,2,A,f,365.363636,893
2,2,1,3,A,f,590.177778,722
3,3,1,4,A,f,1347.285714,605
4,4,1,5,A,f,278.952381,556
...,...,...,...,...,...,...,...
10325,10325,52,694,U,s,112.500000,133
10326,10326,52,758,Y,f,72.214286,598
10327,10327,52,760,Y,f,5208.333333,422
10328,10328,52,761,Y,f,8.000000,146


In [61]:
GA_wkly = GA_wkly.drop(axis = 1, columns = ['Unnamed: 0',])
GA_wkly

,Week,Product,product category,sub-category,Variance,Quantity
0,1,1,A,f,31.125000,515
1,1,2,A,f,365.363636,893
2,1,3,A,f,590.177778,722
3,1,4,A,f,1347.285714,605
4,1,5,A,f,278.952381,556
...,...,...,...,...,...,...
10325,52,694,U,s,112.500000,133
10326,52,758,Y,f,72.214286,598
10327,52,760,Y,f,5208.333333,422
10328,52,761,Y,f,8.000000,146


In [62]:
orders_raw = orders_raw.drop(axis = 1, columns = ['Unnamed: 0'])

NameError: name 'orders_raw' is not defined

In [ ]:
daily_orders_prd = pd.merge(orders_raw, FC_ZIP, on='ZIP_3d')

In [68]:
FC_ZIP[FC_ZIP['ZIP_3d']==292]

,ZIP,City,State,ZIP_3d
292,30336,Atlanta,GA,292


In [64]:
#daily_orders_prd.to_csv('daily_order_prd_22_28')
daily_orders_prd = pd.read_csv('wkly_order_prd_22_28.csv')

In [66]:
len(daily_orders_prd)

63280734

In [65]:
GA = daily_orders_prd[daily_orders_prd['State']=='GA']
GA.head()

,Unnamed: 0,date,Quantity,Product,ZIP_3d,product category,sub-category,ZIP,City,State
3382094,3382094,2023-01-01,0,1,292,A,f,30336,Atlanta,GA
3382095,3382095,2023-01-08,0,1,292,A,f,30336,Atlanta,GA
3382096,3382096,2023-01-15,0,1,292,A,f,30336,Atlanta,GA
3382097,3382097,2023-01-22,0,1,292,A,f,30336,Atlanta,GA
3382098,3382098,2023-01-29,0,1,292,A,f,30336,Atlanta,GA


In [8]:
date = GA['date'].unique().tolist()
wk_num = []
for i in range(len(date)):
    wk_num.append(i%52+1)
    

NameError: name 'GA' is not defined

In [9]:
date_wk = pd.DataFrame(data = {'date':date, 'week':wk_num } )

NameError: name 'date' is not defined

In [8]:
GA_fore_wkly = pd.read_csv('GA_fore_wkly.csv')


,Unnamed: 0,date,Quantity_x,Product,ZIP_3d,product category_x,sub-category_x,ZIP,City,State,Week,product category_y,sub-category_y,Variance,Quantity_y
0,0,2023-01-01,0,1,292,A,f,30336,Atlanta,GA,1,A,f,31.125,515
1,244,2023-01-01,0,1,294,A,f,30336,Atlanta,GA,1,A,f,31.125,515
2,676,2023-01-01,0,1,368,A,f,30336,Atlanta,GA,1,A,f,31.125,515
3,915,2023-01-01,0,1,282,A,f,30336,Atlanta,GA,1,A,f,31.125,515
4,1341,2023-01-01,0,1,293,A,f,30336,Atlanta,GA,1,A,f,31.125,515


In [9]:
#GA_fore_wkly = pd.merge(GA, GA_fore_wkly, on=['date'])
GA_fore_wkly['Week'] = GA_fore_wkly['week']
GA_fore_wkly = GA_fore_wkly.drop(axis = 1, columns = ['week'])


In [10]:
GA_fore_wkly = pd.merge(GA_fore_wkly, GA_wkly, on=['Week', 'Product'])
GA_fore_wkly.columns

Index(['Unnamed: 0', 'date', 'Quantity_x', 'Product', 'ZIP_3d',
       'product category_x', 'sub-category_x', 'ZIP', 'City', 'State', 'Week',
       'product category_y', 'sub-category_y', 'Variance', 'Quantity_y'],
      dtype='object')

In [11]:
GA_fore = GA_fore_wkly[['date', 'Quantity_x', 'Product', 'ZIP_3d', 'product category_x',
       'sub-category_x', 'ZIP', 'City', 'State', 'Week', 'Variance']]




In [12]:
GA_fore

,date,Quantity_x,Product,ZIP_3d,product category_x,sub-category_x,ZIP,City,State,Week,Variance
0,2023-01-01,0,1,292,A,f,30336,Atlanta,GA,1,31.125
1,2023-01-01,0,1,294,A,f,30336,Atlanta,GA,1,31.125
2,2023-01-01,0,1,368,A,f,30336,Atlanta,GA,1,31.125
3,2023-01-01,0,1,282,A,f,30336,Atlanta,GA,1,31.125
4,2023-01-01,0,1,293,A,f,30336,Atlanta,GA,1,31.125
...,...,...,...,...,...,...,...,...,...,...,...
3043142,2028-12-17,0,557,290,R,s,30336,Atlanta,GA,52,288.000
3043143,2028-12-17,0,557,296,R,s,30336,Atlanta,GA,52,288.000
3043144,2028-12-17,0,557,297,R,s,30336,Atlanta,GA,52,288.000
3043145,2028-12-17,0,557,317,R,s,30336,Atlanta,GA,52,288.000


In [13]:
GA_fore = GA_fore.rename(columns={"sub-category_x": "sub-category", "Quantity_x": "Quantity", 'product category_x': 'product category'})



In [5]:
#GA_fore_wkly.to_csv('GA_fore_wkly.csv')
#demo = pd.read_csv('GA_fore_wkly.csv')
GA_fore = pd.read_csv('GA_fore_wkly.csv')
GA_fore

,Unnamed: 0,date,Quantity,Product,ZIP_3d,product category,sub-category,ZIP,City,State,week
0,0,2023-01-01,0,1,292,A,f,30336,Atlanta,GA,1
1,1,2023-01-01,0,2,292,A,f,30336,Atlanta,GA,1
2,2,2023-01-01,0,3,292,A,f,30336,Atlanta,GA,1
3,3,2023-01-01,0,4,292,A,f,30336,Atlanta,GA,1
4,4,2023-01-01,0,5,292,A,f,30336,Atlanta,GA,1
...,...,...,...,...,...,...,...,...,...,...,...
5665811,5665811,2028-12-31,0,368,369,L,f,30336,Atlanta,GA,2
5665812,5665812,2028-12-31,0,372,369,L,m,30336,Atlanta,GA,2
5665813,5665813,2028-12-31,0,500,369,R,f,30336,Atlanta,GA,2
5665814,5665814,2028-12-31,0,501,369,R,f,30336,Atlanta,GA,2


In [12]:
#index = ['Product','ZIP_3d']
#unique_comb = GA_fore.set_index(index).index.unique()

In [17]:
len(GA_fore)%10000

3147

In [ ]:
split_lst = []
for i in range(305):
    split_lst.append(i*10000)
split_lst.append(None)

In [31]:
df_lst = []
for i in range(len(split_lst)-1):
    df_lst.append(GA_fore.loc[split_lst[i]:split_lst[i+1],:].reset_index())

#df1 = df.loc[:5400268,:] # 0 to 5400267 rows and all 60 columns
#df2 = df.loc[5400268:,:] # 5400268 to last rows and all 60 columns


In [32]:
df_lst[304]

,index,date,Quantity,Product,ZIP_3d,product category,sub-category,ZIP,City,State,Week,Variance
0,3040000,2025-12-21,0,653,281,T,f,30336,Atlanta,GA,52,36.2
1,3040001,2025-12-21,0,653,305,T,f,30336,Atlanta,GA,52,36.2
2,3040002,2025-12-21,0,653,309,T,f,30336,Atlanta,GA,52,36.2
3,3040003,2025-12-21,0,653,315,T,f,30336,Atlanta,GA,52,36.2
4,3040004,2025-12-21,0,653,325,T,f,30336,Atlanta,GA,52,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...
3142,3043142,2028-12-17,0,557,290,R,s,30336,Atlanta,GA,52,288.0
3143,3043143,2028-12-17,0,557,296,R,s,30336,Atlanta,GA,52,288.0
3144,3043144,2028-12-17,0,557,297,R,s,30336,Atlanta,GA,52,288.0
3145,3043145,2028-12-17,0,557,317,R,s,30336,Atlanta,GA,52,288.0


## Robust Demand

In [33]:
import math
import time

def Robust_Demand(df_state):
    df_state['Robust Demand'] = ""
    for i in range(len(df_state)):
        exp_d = df_state.loc[i,'Quantity']
        sigma = df_state.loc[i,'Variance']
        if df_state.loc[i,'sub-category']=='f':
            df_state.loc[i,'Robust Demand']= math.ceil(exp_d*(1/7)+2.33*np.sqrt(sigma*(1/7)))
        elif df_state.loc[i,'sub-category']=='m':
            df_state.loc[i,'Robust Demand']= math.ceil(exp_d*(2/7)+2.33*np.sqrt(sigma*(2/7)))
        elif df_state.loc[i,'sub-category']=='s':
            df_state.loc[i,'Robust Demand']= math.ceil(exp_d*(3/7)+2.33*np.sqrt(sigma*(3/7)))
    return df_state

In [35]:
ga2328_lst = []
start_time = time.time()
for i in range(len(df_lst)):
    print(i, " completed","--- %s seconds ---" % (time.time() - start_time))
    ga2328_lst.append(Robust_Demand(df_lst[i]))

0  completed --- 5.817413330078125e-05 seconds ---
1  completed --- 4.947355031967163 seconds ---
2  completed --- 9.788294076919556 seconds ---
3  completed --- 14.606900930404663 seconds ---
4  completed --- 19.303300142288208 seconds ---
5  completed --- 24.261423110961914 seconds ---
6  completed --- 29.23053216934204 seconds ---
7  completed --- 33.966684103012085 seconds ---
8  completed --- 38.66292405128479 seconds ---
9  completed --- 43.66254210472107 seconds ---
10  completed --- 48.848597049713135 seconds ---
11  completed --- 53.55053901672363 seconds ---
12  completed --- 58.272682905197144 seconds ---
13  completed --- 62.97332286834717 seconds ---
14  completed --- 67.98930191993713 seconds ---
15  completed --- 72.64041209220886 seconds ---
16  completed --- 77.26380801200867 seconds ---
17  completed --- 81.94801497459412 seconds ---
18  completed --- 86.87998008728027 seconds ---
19  completed --- 91.72093105316162 seconds ---
20  completed --- 96.47585606575012 seco

169  completed --- 858.9706130027771 seconds ---
170  completed --- 864.1265959739685 seconds ---
171  completed --- 869.1017179489136 seconds ---
172  completed --- 874.0770359039307 seconds ---
173  completed --- 879.0435121059418 seconds ---
174  completed --- 883.9271609783173 seconds ---
175  completed --- 888.8476030826569 seconds ---
176  completed --- 893.7658860683441 seconds ---
177  completed --- 898.9736270904541 seconds ---
178  completed --- 904.503858089447 seconds ---
179  completed --- 910.2536292076111 seconds ---
180  completed --- 915.2279570102692 seconds ---
181  completed --- 920.1688969135284 seconds ---
182  completed --- 925.2037029266357 seconds ---
183  completed --- 930.3467571735382 seconds ---
184  completed --- 935.0399391651154 seconds ---
185  completed --- 940.1022670269012 seconds ---
186  completed --- 945.2400748729706 seconds ---
187  completed --- 950.3168470859528 seconds ---
188  completed --- 955.177716255188 seconds ---
189  completed --- 960

In [36]:
res_df = pd.concat(ga2328_lst)

In [39]:
#res_df.to_csv('2023_28_GA_rob_demand.csv')

## Robust Cap

In [79]:
def Robust_Capacity(df_state):
    df_state['Robust Capacity'] = ""
    for i in range(len(df_state)):
        exp_rd = df_state.loc[i,'Robust Demand']
        if df_state.loc[i,'sub-category']=='f':
            df_state.loc[i,'Robust Capacity']= math.floor((float(exp_rd)*7*275)/(0.9*132000*0.9*0.9*1))
        elif df_state.loc[i,'sub-category']=='m':
            df_state.loc[i,'Robust Capacity']= math.floor((float(exp_rd)*7*556)/(0.9*266880*0.9*0.9*2))
        elif df_state.loc[i,'sub-category']=='s':
            df_state.loc[i,'Robust Capacity']= math.floor((float(exp_rd)*7*320)/(0.9*153600*0.9*0.9*3))
    return df_state

In [106]:
cap_lst = [] 

start_time = time.time()
for i in range(len(ga2328_lst)):
    print(i, " completed","--- %s seconds ---" % (time.time() - start_time))
    cap_lst.append(Robust_Demand(ga2328_lst[i]))

0  completed --- 6.604194641113281e-05 seconds ---
1  completed --- 5.499730825424194 seconds ---
2  completed --- 10.785529851913452 seconds ---
3  completed --- 16.103970050811768 seconds ---
4  completed --- 21.312636852264404 seconds ---
5  completed --- 26.03536581993103 seconds ---
6  completed --- 30.51099705696106 seconds ---
7  completed --- 35.12716484069824 seconds ---
8  completed --- 39.65856194496155 seconds ---
9  completed --- 44.153199911117554 seconds ---
10  completed --- 48.72584891319275 seconds ---
11  completed --- 53.221325159072876 seconds ---
12  completed --- 57.8281090259552 seconds ---
13  completed --- 62.33868908882141 seconds ---
14  completed --- 66.89827585220337 seconds ---
15  completed --- 71.52397394180298 seconds ---
16  completed --- 76.01977801322937 seconds ---
17  completed --- 80.53887796401978 seconds ---
18  completed --- 88.72567772865295 seconds ---
19  completed --- 95.49231171607971 seconds ---
20  completed --- 100.68737387657166 secon

169  completed --- 845.7565131187439 seconds ---
170  completed --- 851.3593158721924 seconds ---
171  completed --- 856.5919909477234 seconds ---
172  completed --- 861.5794229507446 seconds ---
173  completed --- 866.781909942627 seconds ---
174  completed --- 872.0974278450012 seconds ---
175  completed --- 877.4141359329224 seconds ---
176  completed --- 882.9128839969635 seconds ---
177  completed --- 887.744616985321 seconds ---
178  completed --- 892.9763789176941 seconds ---
179  completed --- 898.4774148464203 seconds ---
180  completed --- 903.4125819206238 seconds ---
181  completed --- 908.4202387332916 seconds ---
182  completed --- 913.1061251163483 seconds ---
183  completed --- 918.4672479629517 seconds ---
184  completed --- 923.7814679145813 seconds ---
185  completed --- 929.1614379882812 seconds ---
186  completed --- 934.5912749767303 seconds ---
187  completed --- 940.0881280899048 seconds ---
188  completed --- 945.1405968666077 seconds ---
189  completed --- 949

In [107]:
res_df = pd.concat(cap_lst)

In [108]:
res_df.to_csv('GA_cap_demand_2328.csv')

In [110]:
res_df['Robust Capacity'].unique()

array([0, 1, 2, nan], dtype=object)

# Inventory

In [26]:
inv_df = pd.read_csv('GA_cap_demand_2328.csv')

In [47]:
"""def Inventory(df_state, prod:int):
    prod= df_state[df_state['Product']==prod]
    prod['Cumulative Demand']= prod['Robust Demand'].cumsum()
    
    # Smoothed Cumulative Production
    prod_c=prod.copy().reset_index()
    prod['coeff'] = (prod['Cumulative Demand'].max()-prod['Cumulative Demand'].min())/len(prod)
    prod['Smoothed Cumulative Production']=prod['coeff'].cumsum()
    
    # Fully Smoother Inventory
    prod['Fully Smoother Inventory']=prod['Smoothed Cumulative Production']-prod['Cumulative Demand'] 
    prod=prod.drop(columns='coeff')
    return prod
"""

In [ ]:
""""def Inventory(state:str, prod:int):
    df_state=Robust_Demand(state)
    # generate Cumulative Demand
    prod= df_state[df_state['Product']==prod]
    prod['Cumulative Demand']= prod['Robust Demand'].cumsum()
    
    # Smoothed Cumulative Production
    prod_c=prod.copy().reset_index()
    prod['coeff'] = (prod['Cumulative Demand'].max()-prod['Cumulative Demand'].min())/len(prod)
    prod['Smoothed Cumulative Production']=prod['coeff'].cumsum()
    
    # Fully Smoother Inventory
    prod['Fully Smoother Inventory']=prod['Smoothed Cumulative Production']-prod['Cumulative Demand'] 
    prod=prod.drop(columns='coeff')
    return prod""""

In [115]:
def Inventory(df_state):
    df_state['Robust Inventory']= df_state['Robust Capacity']*50- df_state['Robust Demand']
    INV = [-x if x <0 else 0 for x in df_state['Robust Inventory']]
    INV_int=[]
    for i in INV:
        INV_int.append(math.ceil(i))
    df_state['Robust Inventory']= INV_int
    return df_state

In [117]:
inv_ga = Inventory(inv_df)

In [146]:
inv_ga.columns

Index(['Unnamed: 0', 'index', 'date', 'Quantity', 'Product', 'ZIP_3d',
       'product category', 'sub-category', 'ZIP', 'City', 'State', 'Week',
       'Variance', 'Robust Demand', 'Robust Capacity', 'Robust Inventory'],
      dtype='object')

In [156]:
clean = inv_ga[['date', 'Quantity', 'ZIP_3d', 'sub-category', 'ZIP', 'City', 'State', 'Week',
       'Variance', 'Robust Demand', 'Robust Capacity', 'Robust Inventory']]

In [159]:
fc_demand = clean.groupby(by = ['sub-category', 'ZIP_3d', 'Week'], as_index=False).agg(
{'date' : 'first', 
 'Quantity': 'sum', 
 'ZIP': 'first', 
 'City': 'first', 
 'State': 'first', 
 'Variance': 'first', 
 'Robust Demand': 'sum', 
 'Robust Capacity': 'sum', 
 'Robust Inventory': 'sum'
}
)

#df.groupby(['name'], as_index=False)
#.agg({'value1': 'sum', 'value2': 'sum', 'otherstuff1': 'first', 'otherstuff2': 'first'})




In [160]:
fc_demand

,sub-category,ZIP_3d,Week,date,Quantity,ZIP,City,State,Variance,Robust Demand,Robust Capacity,Robust Inventory
0,f,242,1,2023-01-01,3805,30336,Atlanta,GA,31.125000,9816,0.0,1290
1,f,242,2,2023-01-08,884,30336,Atlanta,GA,46.111111,9963,0.0,0
2,f,242,3,2023-01-15,146,30336,Atlanta,GA,85.300000,7383,0.0,0
3,f,242,4,2023-01-22,162,30336,Atlanta,GA,856.654545,9143,0.0,0
4,f,242,5,2023-01-29,770,30336,Atlanta,GA,1151.181818,6988,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
11512,s,398,45,2023-11-05,0,30336,Atlanta,GA,50.000000,444,0.0,0
11513,s,398,46,2023-11-12,0,30336,Atlanta,GA,180.500000,456,0.0,0
11514,s,398,48,2023-11-26,0,30336,Atlanta,GA,4802.000000,636,0.0,0
11515,s,398,49,2023-12-03,0,30336,Atlanta,GA,18.000000,42,0.0,0


In [25]:
inv_ga[(inv_ga['Product']==1)&(inv_ga['ZIP_3d']==292)&(inv_ga['Product']==1)]

NameError: name 'inv_ga' is not defined

In [33]:
inv_df[(inv_df['Product']==1)&(inv_df['ZIP_3d']==292)&(inv_df['Product']==1)]

,Unnamed: 0,index,date,Quantity,Product,ZIP_3d,product category,sub-category,ZIP,City,State,Week,Variance,Robust Demand,Robust Capacity
0,0,0,2023-01-01,0,1,292,A,f,30336,Atlanta,GA,1,31.125000,5,0.0
70,70,70,2023-12-31,0,1,292,A,f,30336,Atlanta,GA,1,31.125000,5,0.0
140,140,140,2024-12-29,0,1,292,A,f,30336,Atlanta,GA,1,31.125000,5,0.0
210,210,210,2025-12-28,0,1,292,A,f,30336,Atlanta,GA,1,31.125000,5,0.0
280,280,280,2026-12-27,0,1,292,A,f,30336,Atlanta,GA,1,31.125000,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2991256,957,2990957,2024-12-22,0,1,292,A,f,30336,Atlanta,GA,52,521.571429,21,NaN
2991326,1027,2991027,2025-12-21,0,1,292,A,f,30336,Atlanta,GA,52,521.571429,21,NaN
2991396,1097,2991097,2026-12-20,0,1,292,A,f,30336,Atlanta,GA,52,521.571429,21,NaN
2991466,1167,2991167,2027-12-19,0,1,292,A,f,30336,Atlanta,GA,52,521.571429,21,NaN


# Replenish

In [119]:
def Replenish(df):

    Rep = df.groupby(['Week','Product', 'product category','sub-category'])['Robust Inventory'].sum().reset_index()
    Rep = Rep.rename(columns={'Robust Inventory':'Replenishment'})
    return Rep

In [120]:
GA_rep = Replenish(inv_ga)

In [121]:
GA_rep.to_csv('GA_rep.csv')

In [123]:
GA_rep['Replenishment'].unique()

array([ 2986,  9006, 10447, 15488,  6467,  5306,  9473,  8316,     0,
        3652,   742,  2830,  4842,  8611,  4945,  4082,  2702,  4234,
        3821, 12659,  9665,   993,  2906,  3357,  2956,  2224])

# task 3b

In [164]:
GA_fore.columns

Index(['date', 'Quantity', 'Product', 'ZIP_3d', 'product category',
       'sub-category', 'ZIP', 'City', 'State', 'Week', 'Variance'],
      dtype='object')

In [166]:
clean = GA_fore[['date', 'Quantity', 'Product', 'ZIP_3d', 
       'sub-category', 'ZIP', 'City', 'State', 'Week']]
clean

,date,Quantity,Product,ZIP_3d,sub-category,ZIP,City,State,Week
0,2023-01-01,0,1,292,f,30336,Atlanta,GA,1
1,2023-01-01,0,1,294,f,30336,Atlanta,GA,1
2,2023-01-01,0,1,368,f,30336,Atlanta,GA,1
3,2023-01-01,0,1,282,f,30336,Atlanta,GA,1
4,2023-01-01,0,1,293,f,30336,Atlanta,GA,1
...,...,...,...,...,...,...,...,...,...
3043142,2028-12-17,0,557,290,s,30336,Atlanta,GA,52
3043143,2028-12-17,0,557,296,s,30336,Atlanta,GA,52
3043144,2028-12-17,0,557,297,s,30336,Atlanta,GA,52
3043145,2028-12-17,0,557,317,s,30336,Atlanta,GA,52


In [168]:
fc_demand = clean.groupby(by = ['sub-category', 'ZIP_3d', 'Week'], as_index=False).agg(
{'date' : 'first', 
 'Quantity': 'sum', 
 'ZIP': 'first', 
 'City': 'first', 
 'State': 'first'
}
)

In [170]:
fc_demand.to_csv('task3_b.csv')

In [171]:
fc_demand

,sub-category,ZIP_3d,Week,date,Quantity,ZIP,City,State
0,f,242,1,2023-01-01,3805,30336,Atlanta,GA
1,f,242,2,2023-01-08,884,30336,Atlanta,GA
2,f,242,3,2023-01-15,146,30336,Atlanta,GA
3,f,242,4,2023-01-22,162,30336,Atlanta,GA
4,f,242,5,2023-01-29,770,30336,Atlanta,GA
...,...,...,...,...,...,...,...,...
11512,s,398,45,2023-11-05,0,30336,Atlanta,GA
11513,s,398,46,2023-11-12,0,30336,Atlanta,GA
11514,s,398,48,2023-11-26,0,30336,Atlanta,GA
11515,s,398,49,2023-12-03,0,30336,Atlanta,GA


# 10 Scenario Illustration

In [6]:
task3c_df = pd.read_csv("task3c_res_df.csv")

In [11]:
prd_sub = GA_fore[['Product', 'sub-category']].drop_duplicates()
prd_sub

,Product,sub-category
0,1,f
1,2,f
2,3,f
3,4,f
4,5,f
...,...,...
13556,312,s
13559,646,s
17261,224,s
17692,752,s


In [12]:
task3c_df = pd.merge(task3c_df, prd_sub, on = 'Product')

In [15]:
task3c_df = task3c_df.rename(columns = {'Unnamed: 0': 'week'})


In [17]:
task3c_df.columns

Index(['week', 'date', 'Quantity', 'Product', 'ZIP_3d', 'scenario1',
       'scenario2', 'scenario3', 'scenario4', 'scenario5', 'scenario6',
       'scenario7', 'scenario8', 'scenario9', 'scenario10', 'stoc_avg',
       'sum_scene', 'State', 'City', 'sub-category'],
      dtype='object')

In [20]:
fc_demand = task3c_df.groupby(by = ['sub-category', 'ZIP_3d', 'week'], as_index=False).agg(
{'date' : 'first', 
 'Quantity': 'sum', 
 'City': 'first', 
 'State': 'first',
 'scenario1': 'sum',
 'scenario2': 'sum',
 'scenario3': 'sum',
 'scenario4': 'sum',
 'scenario5': 'sum',
 'scenario6': 'sum',
 'scenario7': 'sum',
 'scenario8': 'sum',
 'scenario9': 'sum',
 'scenario10': 'sum',
 'stoc_avg': 'sum'
}
)

In [23]:
fc_demand

,sub-category,ZIP_3d,week,date,Quantity,City,State,scenario1,scenario2,scenario3,scenario4,scenario5,scenario6,scenario7,scenario8,scenario9,scenario10,stoc_avg
0,f,242,0,2023-01-01,0,Atlanta,GA,1221,1193,1485,1500,1144,1150,1413,1238,1480,1420,1306
1,f,242,1,2023-01-08,0,Atlanta,GA,1298,1186,1125,1342,1545,1429,1233,1154,1261,1457,1283
2,f,242,2,2023-01-15,0,Atlanta,GA,1191,1343,1682,1383,1058,1202,1220,1478,1220,1090,1269
3,f,242,3,2023-01-22,79,Atlanta,GA,1251,1337,1280,1633,1258,1306,1235,1310,1313,1293,1306
4,f,242,4,2023-01-29,73,Atlanta,GA,1571,1389,1297,1296,1169,1324,1379,1246,1538,1395,1343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67819,s,398,309,2028-12-03,0,Atlanta,GA,63,38,83,88,48,97,65,47,52,57,63
67820,s,398,310,2028-12-10,0,Atlanta,GA,2,72,44,55,37,36,78,66,55,51,49
67821,s,398,311,2028-12-17,0,Atlanta,GA,74,43,23,65,39,91,32,83,22,87,55
67822,s,398,312,2028-12-24,0,Atlanta,GA,62,67,72,28,44,87,41,49,49,31,52


# Task 4c

In [37]:
rep_dc = pd.read_csv('Rep_DC.csv')

In [38]:
rep_dc = rep_dc.drop(axis = 1, columns = ['Unnamed: 0'])

In [58]:
demo = rep_dc[rep_dc['Product']==1]
len(demo)

19

In [48]:
def trend_generator(rate, length, variation = False, opt=None, cons=None):
    
    '''
    return: 
    if variation = true
    lsts of day trend, day trend opt, day trend cons, day counter
    
    if variation = false
    lsts of day trend, day counter
    '''
    
    if variation == False:
        
        lst_of_dt = []
        counter = []
        for i in range(length):
            lst_of_dt.append(pow(pow(rate, length/365), i/length))
            counter.append(i+1)
        return lst_of_dt, counter

In [172]:
date = [x+1 for x in range(19*7)]
da2024 = date[0:38]
da2025 = date[38:57]
da2028 = date[57:]

In [174]:
len(da2028)
#len(dt2028)

76

In [175]:
len(da2024 + da2025 + da2028)
#len(dt2024 + dt2025 + dt2028)

133

In [176]:
dt2024, counter =  trend_generator(1.5, len(da2024))
dt2025, counter =  trend_generator(2.0/1.5, len(da2025))
dt2028, counter =  trend_generator(1.5, len(da2028))

In [177]:
rep_dc

,20d,Product,product category,sub-category,Demand,Robust Capacity,Robust Inventory
0,1,1,A,f,28402,58,25502
1,1,2,A,f,25973,54,23273
2,1,3,A,f,25480,53,22830
3,1,4,A,f,20605,43,18455
4,1,5,A,f,23405,48,21005
...,...,...,...,...,...,...,...
10388,19,777,Y,s,2196,2,2096
10389,19,778,Y,s,1400,1,1350
10390,19,783,Y,s,1251,1,1201
10391,19,786,Y,s,664,1,614


In [178]:
uni_iter = rep_dc['Product'].unique()


In [239]:
def ramp_demand(prd):
    df = rep_dc[rep_dc['Product']==prd]
    
    if len(df) != 19:
        return None
    dm = df['Demand'].tolist()
    rc = df['Robust Capacity'].tolist()
    ri = df['Robust Inventory'].tolist()
    
    dm24 = dm + dm
    dm28 = dm + dm + dm + dm 
    rc24 = rc + rc
    rc28 = rc + rc + rc + rc
    ri24 = ri + ri
    ri28 = ri + ri + ri + ri
    
    
    dpdm2024 = [int(x*y) for x,y in zip(dt2024,dm24)]
    dpdm2025 = [int(x*y) for x,y in zip(dt2025,[1.5*x for x in dm])]
    dpdm2028 = [int(x*y) for x,y in zip(dt2028,[2*x for x in dm28])]
    
    dprc2024 = [int(x*y) for x,y in zip(dt2024,rc24)]
    dprc2025 = [int(x*y) for x,y in zip(dt2025,[1.5*x for x in rc])]
    dprc2028 = [int(x*y) for x,y in zip(dt2028,[2*x for x in rc28])]
    
    dpri2024 = [int(x*y) for x,y in zip(dt2024,ri24)]
    dpri2025 = [int(x*y) for x,y in zip(dt2025,[1.5*x for x in ri])]
    dpri2028 = [int(x*y) for x,y in zip(dt2028,[2*x for x in ri28])]
    
    res = pd.DataFrame()

    
    res['20_days'] = date
    res['Demand'] = dpdm2024 + dpdm2025 + dpdm2028
    res['Robust Capacity'] = dprc2024 + dprc2025 + dprc2028
    res['Robust Inventory'] = dpri2024 + dpri2025 + dpri2028
    
    return res

In [240]:
df_lst = []
for prd in uni_iter:
    print(prd)
    df = ramp_demand(prd)
    if df is not None:
        df['Product'] = prd
        df_lst.append(df)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
20
21
22
24
27
29
34
55
56
60
61
62
63
64
65
66
67
68
69
70
72
74
75
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
143
144
145
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
165
167
168
169
170
172
173
174
175
176
177
178
181
182
183
185
187
189
191
194
195
196
197
200
208
209
214
216
236
237
240
246
247
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
293
305
306
307
308
309
311
313
314
316
317
319
320
321
322
323
324
325
326
327
328
329
330
333
334
336
337
338
340
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386


In [241]:
res_df_4c = pd.concat(df_lst)

In [251]:
res_df_4c

,20_days,Demand,Robust Capacity,Robust Inventory,Product
0,1,28402,58,25502,1
1,2,34239,70,30735,1
2,3,47909,98,42998,1
3,4,27002,55,24243,1
4,5,38862,79,34894,1
...,...,...,...,...,...
128,129,106797,56,103984,778
129,130,4697,4,4480,778
130,131,14800,8,14367,778
131,132,15965,8,15531,778


# Task 4b

In [245]:
DC_prd = pd.read_csv('DC_20d_prd.csv')

In [248]:
DC_prd = DC_prd.drop(axis = 1, columns = 'Unnamed: 0')

In [250]:
DC_prd

,20d,Product,product category,sub-category,Variance,Quantity,Robust Demand,Robust Capacity,Robust Inventory
0,1,1,A,f,503.109874,22550,31652,65,28402
1,1,2,A,f,479.256168,20638,28973,60,25973
2,1,3,A,f,1580.275659,20203,28430,59,25480
3,1,4,A,f,287.572769,16352,22955,47,20605
4,1,5,A,f,2777.847772,18509,26105,54,23405
...,...,...,...,...,...,...,...,...,...
10388,19,777,Y,s,70.700000,399,2296,2,2196
10389,19,778,Y,s,46.333333,250,1450,1,1400
10390,19,783,Y,s,157.000000,216,1301,1,1251
10391,19,786,Y,s,32.000000,120,714,1,664


In [255]:
def ramp_demand_4b(prd):
    df = rep_dc[rep_dc['Product']==prd]
    
    if len(df) != 19:
        return None
    dm = df['Demand'].tolist()
    rc = df['Robust Capacity'].tolist()
    ri = df['Robust Inventory'].tolist()
    
    dm24 = dm + dm
    dm28 = dm + dm + dm + dm 
    rc24 = rc + rc
    rc28 = rc + rc + rc + rc
    ri24 = ri + ri
    ri28 = ri + ri + ri + ri
    
    
    dpdm2024 = [int(x*y) for x,y in zip(dt2024,dm24)]
    dpdm2025 = [int(x*y) for x,y in zip(dt2025,[1.5*x for x in dm])]
    dpdm2028 = [int(x*y) for x,y in zip(dt2028,[2*x for x in dm28])]
    
    dprc2024 = [int(x*y) for x,y in zip(dt2024,rc24)]
    dprc2025 = [int(x*y) for x,y in zip(dt2025,[1.5*x for x in rc])]
    dprc2028 = [int(x*y) for x,y in zip(dt2028,[2*x for x in rc28])]
    
    dpri2024 = [int(x*y) for x,y in zip(dt2024,ri24)]
    dpri2025 = [int(x*y) for x,y in zip(dt2025,[1.5*x for x in ri])]
    dpri2028 = [int(x*y) for x,y in zip(dt2028,[2*x for x in ri28])]
    
    res = pd.DataFrame()

    
    res['20_days'] = date
    res['Demand'] = dpdm2024 + dpdm2025 + dpdm2028
    res['Robust Capacity'] = dprc2024 + dprc2025 + dprc2028
    res['Robust Inventory'] = dpri2024 + dpri2025 + dpri2028
    
    inv = dpri2024 + dpri2025 + dpri2028
    sum_scene = [0 for x in inv]
    
    for i in range(10):
        col_name = 'Capacity_Scenario_'
        col_name = col_name + str(i+1)
        scene = [x + np.random.randint(-int(x/3), int(x/3)+1) if x>0 else x + np.random.randint(max(demand)/3) for x in inv]
        res[col_name] = scene
        sum_scene = [x+y for x,y in zip(sum_scene, scene)]
        
    res['stoc_avg'] = [int(x/10) for x in sum_scene]
    res['sum_scene'] = sum_scene

    return res



In [257]:
df_lst = []
for prd in uni_iter:
    print(prd)
    df = ramp_demand_4b(prd)
    if df is not None:
        df['Product'] = prd
        df_lst.append(df)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
20
21
22
24
27
29
34
55
56
60
61
62
63
64
65
66
67
68
69
70
72
74
75
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
143
144
145
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
165
167
168
169
170
172
173
174
175
176
177
178
181
182
183
185
187
189
191
194
195
196
197
200
208
209
214
216
236
237
240
246
247
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
293
305
306
307
308
309
311
313
314
316
317
319
320
321
322
323
324
325
326
327
328
329
330
333
334
336
337
338
340
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386


In [258]:
res_4b_df = pd.concat(df_lst)

In [260]:
res_4b_df.to_csv('res_4b_df.csv')

In [261]:
res_4b_df

,20_days,Demand,Robust Capacity,Robust Inventory,Capacity_Scenario_1,Capacity_Scenario_2,Capacity_Scenario_3,Capacity_Scenario_4,Capacity_Scenario_5,Capacity_Scenario_6,Capacity_Scenario_7,Capacity_Scenario_8,Capacity_Scenario_9,Capacity_Scenario_10,stoc_avg,sum_scene,Product
0,1,28402,58,25502,31066,26047,23716,26832,20230,23730,31633,32545,23086,17341,25622,256226,1
1,2,34239,70,30735,23787,35791,33332,22266,37241,27965,28810,21687,24731,33683,28929,289293,1
2,3,47909,98,42998,40495,39190,29571,49285,36923,29009,47578,54694,40745,40839,40832,408329,1
3,4,27002,55,24243,28183,26267,22405,21289,20192,28317,19626,26323,24753,17654,23500,235009,1
4,5,38862,79,34894,44290,42658,36737,28476,33378,32203,36093,45919,31015,34578,36534,365347,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,129,106797,56,103984,127207,69869,80653,71827,109029,87572,105030,91342,102794,114493,95981,959816,778
129,130,4697,4,4480,3278,5915,4079,5711,4300,5835,4998,4943,3826,4930,4781,47815,778
130,131,14800,8,14367,17624,16527,18624,14293,9807,18098,11448,18209,18899,9735,15326,153264,778
131,132,15965,8,15531,20243,19512,13027,16573,14407,12899,10748,19719,17847,12930,15790,157905,778


In [ ]:
def sto_generator(prd, zip3, num_scene):
    df = ramp_demand(prd,zip3)
    df['Product'] = prd
    df['ZIP_3d'] = zip3
    demand = df['Quantity'].to_list()
    
    
    
    return df, col_lst
    
    
